# 2.1 Instruction Formatting Part 2
This is due to my local machine not being able to download unsloth, hence, this notebook was done on kaggle and imported to my local for formatting

## Setting Up Unsloth

In [13]:
import numpy as np
import pandas as pd

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/combined-data/combined_data.csv
/kaggle/input/combined-val-data/combined_val_data.csv


In [14]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 7.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 57.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 92.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.9 MB/s eta 

In [19]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Data Prep

In [15]:
df = pd.read_csv("../input/combined-data/combined_data.csv")

df.head()

def convert_convo(df):
    conversations = []
    
    for _, row in df.iterrows():
        conversation = [
            {"from": "human", "value": row['user_msg']},
            {"from": "gpt", "value": row['output']}
        ]
        conversations.append(conversation)
    
    return pd.DataFrame({'conversations': conversations})

df = convert_convo(df)
df.head()

,conversations
0,"[{'from': 'human', 'value': '""về trễ ."" Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}. Đây là chủ đề: lecturer'}, {'from': 'gpt', 'value': 'negative'}]"
1,"[{'from': 'human', 'value': '""US Stocks Rise, but Growth Concerns Linger; Market Gains 0.2% Amid Ongoing Economic Uncertainty and Global Trade Tensions"" Base on the sentiment of the headline, classify it as neutral, positive, or negative.'}, {'from': 'gpt', 'value': 'neutral'}]"
2,"[{'from': 'human', 'value': '""cần đa dạng hóa bài tập hơn ."" Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}. Đây là chủ đề: training_program'}, {'from': 'gpt', 'value': 'negative'}]"
3,"[{'from': 'human', 'value': '""The smart money may be bailing out of Amazon, but that doesn't mean that you have to drop your AMZN stock, too. It depends on your plan."" What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mi..."
4,"[{'from': 'human', 'value': '""nội dung môn học em nghĩ là rộng quá ạ ."" Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}. Đây là chủ đề: training_program'}, {'from': 'gpt', 'value': 'negative'}]"


In [16]:
from pathlib import Path

output_dir = Path("output")
output_dir.mkdir(exist_ok=True)
df.to_csv(output_dir / "combined_data_convo.csv", index=False)


### 3 examples in conversation form

In [17]:
pd.set_option('display.max_colwidth', 300)
df.head(3)

,conversations
0,"[{'from': 'human', 'value': '""về trễ ."" Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}. Đây là chủ đề: lecturer'}, {'from': 'gpt', 'value': 'negative'}]"
1,"[{'from': 'human', 'value': '""US Stocks Rise, but Growth Concerns Linger; Market Gains 0.2% Amid Ongoing Economic Uncertainty and Global Trade Tensions"" Base on the sentiment of the headline, classify it as neutral, positive, or negative.'}, {'from': 'gpt', 'value': 'neutral'}]"
2,"[{'from': 'human', 'value': '""cần đa dạng hóa bài tập hơn ."" Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}. Đây là chủ đề: training_program'}, {'from': 'gpt', 'value': 'negative'}]"


In [21]:
from datasets import load_dataset, Dataset
from unsloth.chat_templates import get_chat_template
import pandas as pd


tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def format_prompts(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = Dataset.from_pandas(df)

In [22]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(format_prompts, batched = True,)

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/6500 [00:00<?, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

In [31]:
from pathlib import Path

# ensure output folder
out_dir = Path("/kaggle/working")
out_dir.mkdir(exist_ok=True)

pd_df = dataset.to_pandas()
pd_df.to_csv(out_dir / "formatted_prompts.csv", index=False, encoding="utf-8-sig")

In [29]:
for i in range(3):
    print("Example",i)
    print("-" * 9)
    print(dataset[i]["text"])
    print("")
    print("-" * 20)

Example 0
---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

"về trễ ." Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}. Đây là chủ đề: lecturer<|eot_id|><|start_header_id|>assistant<|end_header_id|>

negative<|eot_id|>

--------------------
Example 1
---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

"US Stocks Rise, but Growth Concerns Linger; Market Gains 0.2% Amid Ongoing Economic Uncertainty and Global Trade Tensions" Base on the sentiment of the headline, classify it as neutral, positive, or negative.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

neutral<|eot_id|>

--------------------
Example 2
---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|